# The BrieCoupler class: connecting Barrier3d models through alongshore sediment transport

In this notebook we illustrate the functionality of the BeachDuneManager module within `CASCADE`. In the BeachDuneManager module, we incorporate beach nourishment and dune management strategies to maintain a coastal community at a fixed cross-shore position. Specifically, we: 
- Widen the beach via nourishment of the shoreface
- Nourish dunes or rebuild following storms
- Remove overwash to account for the filtering effect of development on overwash deposition (Rogers et al., 2015) and place it back on the shoreface
- Remove overwash to account for bulldozing and place back on the adjacent dunes

These management strategies are employed until the barrier reaches a minimum width and can no longer sustain a community, here defined as the combined width of a single roadway and building footprint (50 m). Thereafter, the community is abandoned and the barrier is allowed to evolve in accordance with the rules and dynamics in Barrier3D. 

For a complete description of the dynamics in the BeachDuneManager module, we direct the user to Anarde et al., (). Here, we seek only to hold your hand and walk you through use of the BeachDuneManager.  

In [ ]:
import os
import sys
import time
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# sys.path.extend(['/Users/KatherineAnardeWheels/PycharmProjects/Barrier3D', '/Users/KatherineAnardeWheels/PycharmProjects/brie'])
datadir = "../cascade/data/"
Path(datadir)
notebook_dir = "../notebooks/"

from cascade import Cascade
from cascade.tools import plotters as cascade_plt

In [ ]:
alongshore_section_count = 6
background_erosion = [-1] * alongshore_section_count
nt = 100
roadway_management_on = [True] * 3 + [False] * 3
beach_nourishment_on = [False] * 3 + [True] * 3

cascade = Cascade(
    datadir,
    name="AST_demonstration",
    wave_height=1,
    wave_period=7,
    wave_asymmetry=0.8,
    wave_angle_high_fraction=0.2,
    sea_level_rise_rate=0.004,
    sea_level_rise_constant=True,
    background_erosion=background_erosion,
    alongshore_section_count=alongshore_section_count,
    time_step_count=nt,
    num_cores=1,
    roadway_management_module=roadway_management_on,  # no roadway management
    alongshore_transport_module=True,  # couple brie
    beach_nourishment_module=beach_nourishment_on,
    community_economics_module=False,  # no community dynamics
    nourishment_interval=10,  # yrs
    nourishment_volume=100,  # m^3/m
    overwash_filter=90,  # % overwash removed
    overwash_to_dune=10,
)

# Loop for 50 years at a 10 year interval, 100 m^3/m and then 50 years at a 20 year interval with 300 m^3/m
nt = 50
for _ in tqdm(range(nt - 1)):
    cascade.update()
    if cascade.b3d_break:
        break

# # during the CASCADE initialization, the nourishment interval and volume is specified individually for each
# # barrier3d alongshore cell; so to update these values, we need to specify which barrier3d cell we want to modify
# # (here, we only have one cell)
# for
# cascade.nourishment_interval[iB3D] = 20  # increase to 20 years
# cascade.nourishment_volume[iB3D] = 300  # increase to 300 m^3/m

# for _ in tqdm(range(nt)):
#     cascade.update()
#     if cascade.road_break or cascade.b3d_break:
#         break